# README
* Please do not run anything in section 3.2.3 (BERT section)
* Please run sections 0, 1, 2, 3.1, 3.2.1, 3.2.2, 3.3, 3.4, 4, 5.0 sequentially first before running section 5.
* If you run any parts of section 5.3-5.7, make sure you run the first cell under the subsection heading that sets the embedding matrix variable.
* The model design of the best model is in 4.3.1 (in the title), we did not create a specific section for this to make the code running to be more smooth.
* Evaluation result per ablation study is shown at the end as a markdown table from train model results logs shown.

# 0 - Dependencies

In [ ]:
!pip install -U 'scikit-learn<0.24'
!pip install urllib3==1.25.11
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

import numpy as np
# Word Embeddings (Word2Vec), this should be eval'ed later for which one
# gensim models
from gensim.models import Word2Vec
from gensim.models import FastText
# for google saving
from google.colab import drive
# less warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import json

##### Importing librarys
import re
from tqdm.notebook import tqdm
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
from nltk.stem.porter import *
from nltk.stem.lancaster import *
nltk.download('wordnet')
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import pickle
!pip install contractions
import contractions
# For implementing the word2vec family of algorithms
from gensim.models import Word2Vec
from prettytable import PrettyTable

!pip install wikipedia
import wikipedia
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

from tqdm import tqdm
!pip install transformers
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import Dataset, DataLoader

import nltk
# pos tag import and use
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch
import torch.nn as nn
#from google.colab import drive
#drive.mount('/content/drive')
#%cd drive/MyDrive/COMP5046_asm2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 4.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 4.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successf

# 1 - Data Download and Load

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download
id = '1XJgfU-HEMbGMtq7r-V_Sc7V54wM8zwlM'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')

id = '1BjMim7rG-6syYJts2Ghjk4elSxH-mQbi'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('val.csv')

id = '1-7JszptC6yhU9PHegI9gPqFR1TnV-mtU'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test_without_labels.csv')

train_data = pd.read_csv("/content/train.csv")
val_data = pd.read_csv("/content/val.csv")
test_data = pd.read_csv("/content/test_without_labels.csv")

# make into list
train_sents = train_data['sents'].tolist()
train_labels = train_data['labels'].tolist()
val_sents = val_data['sents'].tolist()
val_labels = val_data['labels'].tolist()
test_sents = test_data['sents'].tolist()

# 2 - Data Preprocessing

## 2.1 - Convert to lower case

In [ ]:
def to_lower(sents):
    new_sents = []
    for sent in sents:
        new_sents.append(sent.lower())
    return new_sents

In [ ]:
train_sents = to_lower(train_sents)
val_sents = to_lower(val_sents)
test_sents = to_lower(test_sents)
all_sents = train_sents + val_sents + test_sents

## 2.2 - Tokenisation

In [ ]:
def tokenisation(sents):
    tokenised_sents = []
    for sent in tqdm(sents):
        tokenised_sents.append(sent.split(' '))
    return tokenised_sents

In [ ]:
train_data = tokenisation(train_sents)
target_y_train = tokenisation(train_labels)
validation_data = tokenisation(val_sents)
target_y_validation = tokenisation(val_labels)
test_data = tokenisation(test_sents)
all_data = train_data + validation_data + test_data

100%|██████████| 500/500 [00:00<00:00, 696265.60it/s]


## 2.3 - Word list, tag dicts

In [ ]:
word_to_ix = {}
for sentence in all_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())
word_to_id = {word_list[i]: i for i in range(len(word_list))}
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in target_y_train+target_y_validation:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)
ix_to_tag = {v:k for k, v in tag_to_ix.items()}

## 2.4 - Prepare inputs and labels

### 2.4.1 - Convert data to ids

In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

In [ ]:
train_input_index =  to_index(train_data,word_to_ix)
train_output_index = to_index(target_y_train,tag_to_ix)
val_input_index = to_index(validation_data,word_to_ix)
val_output_index = to_index(target_y_validation,tag_to_ix)
test_input_index = to_index(test_data,word_to_ix)

### 2.4.2 - Convert labels to ids

In [ ]:
def convert_labels_to_ids(tag_to_ix, labels_ls):
    new_labels = []
    for labels in tqdm(labels_ls):
        new_labels.append([tag_to_ix[tag] for tag in labels])
    return new_labels

In [ ]:
train_labels = convert_labels_to_ids(tag_to_ix, tokenisation(train_labels))
val_labels = convert_labels_to_ids(tag_to_ix, tokenisation(val_labels))

100%|██████████| 8705/8705 [00:00<00:00, 600912.05it/s]


# 3 - Input Embedding

## 3.1 - Aspect 1: Syntactic Textual Feature Embedding

### POS Tagging

In [ ]:
def get_tags(sents):
    tagged_sents = []
    for tokens in tqdm(sents):
        tagged_tups = pos_tag(tokens)
        tagged_sents.append([item[1] for item in tagged_tups])
    return tagged_sents
    
def prepare_pos(train_sents, val_sents, test_sents):
    # obtain pos tags for all sents
    tagged_train_sents = get_tags(tokenisation(train_sents))
    tagged_val_sents = get_tags(tokenisation(val_sents))
    tagged_test_sents = get_tags(tokenisation(test_sents))

    tagged_sents = tagged_train_sents + tagged_val_sents + tagged_test_sents

    # obtain all different tags
    tags = set([])
    for ts in tqdm(tagged_sents):
        for t in ts:
            tags.add(t)

    # construct tag2index (key: tags, value: index) for embedding
    postag2index = {t: i + 5 for i, t in enumerate(list(tags))}
    postag2index['[OOV]'] = 0  # The special value used to tag OOVs
    postag2index['[CLS]'] = 1  # The special value used to tag START
    postag2index['[SEP]'] = 2  # The special value used to tag END
    postag2index['[PAD]'] = 3  # The special value used to tag PAD
    postag2index['[sepa]'] = 4  # The special value used to tag SEPA


    def pos_embeeding(tagged_sents):
        pos_embeeding_sents = []
        for tags in tqdm(tagged_sents):
            temp = [postag2index[tag] if tag in postag2index else postag2index['[OOV]'] for tag in tags]
            pos_embeeding_sents.append(temp)
        return pos_embeeding_sents
    
    pos_emb_train = pos_embeeding(tagged_train_sents)
    pos_emb_val = pos_embeeding(tagged_val_sents)
    pos_emb_test = pos_embeeding(tagged_test_sents)

    return postag2index, tags, pos_emb_train, pos_emb_val, pos_emb_test, tagged_train_sents, tagged_val_sents, tagged_test_sents

postag2index, tags, pos_emb_train, pos_emb_val, pos_emb_test, tagged_train_sents, tagged_val_sents, tagged_test_sents = prepare_pos(train_sents, val_sents, test_sents)

100%|██████████| 500/500 [00:00<00:00, 400219.85it/s]


## 3.2 - Aspect 2: Semantic Textual Feature Embedding

### 3.2.1 Glove-twitter-25

In [ ]:
import gensim.downloader as api
gt25 = api.load("glove-twitter-25")

embedding_matrix_gt25 = []
for word in word_list:
    if word in gt25.wv.vocab:
        embedding_matrix_gt25.append(gt25.wv[word])
    else:
        embedding_matrix_gt25.append([0]*25)
embedding_matrix_gt25 = np.array(embedding_matrix_gt25)

[==================================================] 100.0% 104.8/104.8MB downloaded


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


### 3.2.2 - Word2Vec

In [ ]:
ttv_wvsg = Word2Vec(
    sentences= all_data, 
    size=100, window=2, min_count=0, workers=2, sg=1)

embedding_matrix_wvsg = []
for word in word_list:
    embedding_matrix_wvsg.append(ttv_wvsg.wv[word])
embedding_matrix_wvsg = np.array(embedding_matrix_wvsg)

# a smaller matrix that we used in best model, the one above is for ablation studies

# this is word2vec cbow
ttv_wvsg_smaller = Word2Vec(
    sentences= all_data, 
    size=50, window=2, min_count=0, workers=2, sg=1)
embedding_matrix_wvsg_smaller = []
for word in word_list:
    embedding_matrix_wvsg_smaller.append(ttv_wvsg_smaller.wv[word])
embedding_matrix_wvsg_smaller = np.array(embedding_matrix_wvsg_smaller)

### 3.2.3 - BERT [Please do not run]

#### BERT Tokenisation

In [ ]:
def bert_padding(sents, max_length = 0):
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    max_sent = []
    for tokens in sents:
        length = len(tokens)
        if max_length < length:
            max_length = length
            max_sent = tokens
    print('max length is ', max_length)
    print(max_sent)
    print(bert_tokenizer.decode(bert_tokenizer.convert_tokens_to_ids(max_sent)))

    new_sents = []
    for tokens in tqdm(sents):
        padded_tokens = tokens +['[PAD]' for _ in range(max_length - len(tokens))]
        new_sents.append(padded_tokens)
    return new_sents


def bert_attention_mask(sents):
    new_sents = []
    for tokens in tqdm(sents):
        attn_mask = [ 1 if token != '[PAD]' else 0 for token in tokens]
        new_sents.append(attn_mask)
    return new_sents

def bert_token_type_id(sents):
    new_sents = []
    for tokens in tqdm(sents):
        seg_ids = [0 for _ in range(len(tokens))]
        new_sents.append(seg_ids)
    return new_sents

def collect_new_tokens(original, decoded):
    new_token_list = set()
    decoded_idx = 0
    for original_idx in range(len(original)):
        if original[original_idx] == decoded[decoded_idx]:
            decoded_idx += 1
        else:
            # we want decode[decoded_idx:?] == original[original_idx]
            new_token = decoded[decoded_idx]
            for j in range(decoded_idx + 1, len(decoded)):
                new_token += decoded[j]
                if new_token == original[original_idx]:
                    decoded_idx = j + 1
                    break
            new_token_list.add(new_token)
    return list(new_token_list)

def identify_new_tokens(sents):

    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    print('Tokeniser has a vocab of size {}'.format(len(bert_tokenizer)))
    bert_tokenizer.add_tokens(['[sepa]'], special_tokens=True)
    tokenised_sents = []
    original = []
    for sent in tqdm(sents):
        tokens = bert_tokenizer.tokenize(sent)
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        original.append(['[CLS]'] + sent.split(' ') + ['[SEP]'])
        tokenised_sents.append(tokens)

    i = 0
    new_tokens = []
    for sent in tqdm(tokenised_sents):
        bert_id = bert_tokenizer.convert_tokens_to_ids(sent)
        decoded = bert_tokenizer.decode(bert_id)
        if len(decoded.split(' ')) != len(original[i]):
            new_tokens += collect_new_tokens(original[i], decoded.split(' '))
        i += 1
    return bert_tokenizer, set(new_tokens)

def bert_tokenisation(sents, bert_tokenizer):
    
    tokenised_sents = []
    for sent in tqdm(sents):
        tokens = bert_tokenizer.tokenize(sent)
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        tokenised_sents.append(tokens)
        
    sents_id = []
    for sent in tqdm(tokenised_sents):
        bert_id = bert_tokenizer.convert_tokens_to_ids(sent)
        sents_id.append(bert_id)
    attention_mask = bert_attention_mask(tokenised_sents)
    token_type_id = bert_token_type_id(tokenised_sents)
    
    return sents_id, attention_mask, token_type_id, tokenised_sents

#### Add New Tokens

In [ ]:
BERT_TOKENISER, new_tokens = identify_new_tokens(all_sents)
print(new_tokens)
new_tokens.remove('[CLS].[SEP]')
new_tokens.remove(":'d[SEP]")
new_tokens.add(":'")
new_tokens.add("1.")
new_tokens.add(";!")
new_tokens.add("t.")
print(new_tokens)
BERT_TOKENISER.add_tokens(list(new_tokens), special_tokens=False)

In [ ]:
train_sents_id, train_attention_mask, train_token_type_id, bert_tokenised_sents_train = bert_tokenisation(train_sents, BERT_TOKENISER)
val_sents_id, val_attention_mask, val_token_type_id, bert_tokenised_sents_val = bert_tokenisation(val_sents, BERT_TOKENISER)
test_sents_id, test_attention_mask, test_token_type_id, bert_tokenised_sents_test = bert_tokenisation(test_sents, BERT_TOKENISER)

#### Generate Masks

In [ ]:
def bert_mask(tokenised_bert):
    mask_ls = []
    for i in tqdm(range(len(tokenised_bert))):
        mask = []
        sents = tokenised_bert[i]
        for j in range(len(sents)):
            token = sents[j]
            if token == '[CLS]':
                mask.append(-101)
            elif token == '[SEP]':
                mask.append(-102)
            elif token == '[PAD]':
                mask.append(-100)
            elif token == '[sepa]':
                mask.append(-1)
                continue
            elif (len(token) > 2 and token[:2] == "##"):
                mask.append(0)
            else:
                mask.append(1)
        assert len(mask) == len(sents)
        mask_ls.append(mask)
    return mask_ls

In [ ]:
bert_mask_train = bert_mask(bert_tokenised_sents_train)
bert_mask_val = bert_mask(bert_tokenised_sents_val)
bert_mask_test = bert_mask(bert_tokenised_sents_test)

#### Obtain BERT Embeddings

In [ ]:
def get_embeddings(bert_sents_id, bert_attention_mask, bert_token_type_id, mask):
    bert = BertModel.from_pretrained('bert-base-uncased')
    bert.cuda()
    weights = bert.embeddings.word_embeddings.weight.data
    new_weights = weights
    print(new_tokens)
    for i in list(new_tokens)+['[sepa]']:
        new_weights = torch.cat((new_weights, weights[101:102]), 0)
    new_emb = nn.Embedding.from_pretrained(new_weights, padding_idx=0, freeze=False)
    bert.embeddings.word_embeddings = new_emb
    for param in bert.parameters():
        param.requires_grad = False

    all_emb = []
    new_mask =[]
    for m in mask:
        new_mask.append(torch.tensor(m[1:-1]).cuda())
    mask = new_mask

    for i in tqdm(range(len(bert_sents_id))):

        # Obtain BERT embedding
        sent_id = torch.tensor(bert_sents_id[i]).unsqueeze(0).cuda()
        attention_mask = torch.tensor(bert_attention_mask[i]).unsqueeze(0).cuda()
        token_type_id = torch.tensor(bert_token_type_id[i]).unsqueeze(0).cuda()

        outputs = bert(sent_id, attention_mask=attention_mask, token_type_ids = token_type_id)
        hidden_state = outputs[0]
        bert_emb = hidden_state.squeeze()
        emb = bert_emb[1:-1, :].cuda()

        masked_emb = emb[mask[i][:] != 0]
        all_emb.append(masked_emb)
    return all_emb

In [ ]:
embeddings_train = get_embeddings(train_sents_id, train_attention_mask, train_token_type_id, bert_mask_train)
embeddings_val = get_embeddings(val_sents_id, val_attention_mask, val_token_type_id, bert_mask_val)
embeddings_test = get_embeddings(test_sents_id, test_attention_mask, test_token_type_id, bert_mask_test)
embeddings = embeddings_train + embeddings_val + embeddings_test

#### Convert to Embedding Matrix

In [ ]:
word_emb_dict = {}

for i in range(len(all_data)):
    sent = all_data[i]
    emb_set = embeddings[i]
    for j in range(len(sent)):
        word = sent[j]
        if word in word_emb_dict.keys():
            word_emb_dict[word] = (word_emb_dict[word] + emb_set[j]) / 2
        else:
            word_emb_dict[word] = emb_set[j]
            

embedding_matrix_bert = []
for i in range(0, len(word_list)):
    word = word_list[i]
    embedding_matrix_bert.append(word_emb_dict[word])

In [ ]:
torch.save(embedding_matrix_bert, 'embedding_matrix_bert.pt')

In [ ]:
embedding_matrix_bert = torch.load("embedding_matrix_bert.pt")
embedding_matrix_bert = [i.cpu().tolist() for i in embedding_matrix_bert]
embedding_matrix_bert = np.array(embedding_matrix_bert)
print(embedding_matrix_bert.shape)
EMBEDDING_DIM = embedding_matrix_bert.shape[1]
print(EMBEDDING_DIM)

## 3.3 - Aspect 3: Domain Feature Embedding

### 3.3.1 Wiki - Glossary of video game terms

In [ ]:
doc = wikipedia.page("Glossary_of_video_game_terms").content

clean_doc = re.sub(r'[^\w\s]','',doc)
tokenized_doc = word_tokenize(clean_doc)
lower_tokens = [t.lower() for t in tokenized_doc]
sww = sw.words("english")
tokenized_doc = [w for w in lower_tokens if not w in sww]

# pass to w2v
wiki_w2v = Word2Vec(
    sentences=tokenized_doc, 
    size=50, window=4, min_count=0, workers=2, sg=0)

embedding_matrix_wiki = []
emb_dim2 = wiki_w2v.vector_size
occ = 0
for word in word_list:
    if word in wiki_w2v.wv:
        occ += 1
        embedding_matrix_wiki.append(wiki_w2v.wv[word])
    else:
        embedding_matrix_wiki.append([0]*emb_dim2)
embedding_matrix_wiki = np.array(embedding_matrix_wiki)

print("we have {} words".format(len(word_list)))
print("{} words are in dota dataset".format(occ))

we have 11243 words
36 words are in dota dataset


### 3.3.2 - Dota chat dataset

In [ ]:
# from https://www.kaggle.com/code/mammykins/dota-2-allchat-wordcloud/data

drive = GoogleDrive(gauth)
id = "19jsNwQ5vn5TfrskwNiQRbFkJevjvGKH8"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('chat.csv')

dota = pd.read_csv("/content/chat.csv")
data = dota['key'].tolist()

print(data[0:10])

##### 1.1 URL Removal
def remove_url(sentences):
    sentences_length = []
    sentences_wo_url = []
    for sentence in tqdm(sentences, desc="Removing URL"):
        # removing links ending with ||| from text data
        sentence = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*\|{3}', '', str(sentence))
        # removing links of other forms from text data
        sentence = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(sentence))
        sentences_length.append(len(sentence.split()))
        sentences_wo_url.append(sentence)
    return sentences_wo_url


##### 1.2.1 Case-folding
# To convert each sentence to lowercase
def to_lowercase(sentences):
    sentences_lower = []
    for sentence in tqdm(sentences, desc="Converting lowercase"):
        sentence = sentence.lower()
        sentences_lower.append(sentence)
    return sentences_lower


##### 1.2.2 Punctuation removal
# To remove any punctuation
def remove_punctuation(sentences):
    sentences_wo_punc = []
    for sentence in tqdm(sentences, desc="Removing punctuation"):
        # Take into account '|||' situation, hence, replace punctuation with space instead of empty string
        sentence = re.sub(r'[^\w\s]',' ',str(sentence))
        sentences_wo_punc.append(sentence)
    return sentences_wo_punc


##### 1.2.3 Numbers Removal
# To remove any number that is not a part of a word
def remove_numbers(sentences):
    sentences_wo_num = []
    for sentence in tqdm(sentences, desc="Removing numbers"):
        sentence = re.sub("(\s\d+)",'', str(sentence))
        sentences_wo_num.append(sentence)
    return sentences_wo_num


##### 1.2.4 Contractions expansion
# To expand contractions in the sentence
def remove_contractions(sentences):
    sentences_wo_contraction = []
    for sentence in tqdm(sentences, desc="Expanding contractions"):
        sentence = contractions.fix(sentence)
        sentences_wo_contraction.append(sentence)
    return sentences_wo_contraction

##### 1.2.6 Stopwords removal
def remove_stopwords(sentences):
    stop_words = sw.words('english')
    sentences_wo_sw = []
    for tokens in tqdm(sentences, desc="Removing stopwords"):
        filtered_sentence = [w for w in tokens if not w in stop_words]
        sentences_wo_sw.append(filtered_sentence)
    return sentences_wo_sw



##### 1.2.9 Preprocessing function
def data_preprocessing(datasets: list, config):
    '''
    To condcuct each data preprocessing method based on the configuration, the
    case-folding, punctuation removal and tokenisation are default techniques to
    be applied

    Arguments:
    datasets -- a list of data to be pre-processed
    config -- a list of data preprocessing techniques to be applied
    
    Returns:
    processed_datasets -- a list of data that is pre-processed
    '''
    print("Start data preprocessing-------------------------")
    config = set(config)
    processed_datasets = []
    for dataset in datasets:
        # I believe that methods for data preprocessing need to be sequential 
        # These preprocessings are happened before tokenisation
        config_set = config.copy()
        if 'url' in config_set:
            config_set.remove('url')
            processed_sentences = remove_punctuation(to_lowercase(remove_url(dataset)))
        else:
            processed_sentences = remove_punctuation(to_lowercase(dataset))
        if 'number' in config_set:
            config_set.remove('number')
            processed_sentences = remove_numbers(processed_sentences)
        if 'contraction' in config_set:
            config_set.remove('contraction')
            processed_sentences = remove_contractions(processed_sentences)

        # Tokenisation
        processed_sentences = tokenisation(processed_sentences)

        # These preprocessings are happended after tokenisation 
        if 'stopword' in config_set:
            config_set.remove('stopword')
            processed_sentences = remove_stopwords(processed_sentences)
        if len(config_set):
            # remaining methods are not defined
            raise NameError('Data preprocessing methods {} not defined'.format(config_set))
        processed_datasets.append(processed_sentences)
    print("Done!")
    return processed_datasets


config1 = ['url', 'number', 'contraction', 'stopword']
dota = data_preprocessing([data], config1)
dota_raw = dota[0]
print(dota_raw[0:10])
new_dota = []
for i in tqdm(dota_raw):
    new_i = []
    for t in i:
        if not(t == '' or t =='\n' or t == '\n\n'):
            new_i.append(t)
    new_dota.append(new_i)
print(new_dota[0:10])
embedding_matrix_dota = Word2Vec(
sentences= new_dota, 
size=50, window=1, min_count=0, workers=2, sg=1)

#torch.save(embedding_matrix_dota, "/content/drive/MyDrive/COMP5046 A2/embedding_matrix_dota_chat.model")

['force it', 'space created', 'hah', 'ez 500', 'mvp ulti', 'bye', 'hah', 'fate', 'is cruel', 'fuck my ass']
Start data preprocessing-------------------------


Removing stopwords: 100%|██████████| 1439488/1439488 [00:12<00:00, 116779.19it/s]


Done!
[['force'], ['space', 'created'], ['hah'], ['ez'], ['mvp', 'ulti'], ['bye'], ['hah'], ['fate'], ['cruel'], ['fuck', 'ass']]


100%|██████████| 1439488/1439488 [00:03<00:00, 469249.39it/s]


[['force'], ['space', 'created'], ['hah'], ['ez'], ['mvp', 'ulti'], ['bye'], ['hah'], ['fate'], ['cruel'], ['fuck', 'ass']]


## 3.4 - Build Embedding matrix

In [ ]:
EMBEDDING_DIM=0
embedding_matrix = []
occ = 0
occ2 = 0
for i in range(0, len(word_list)):
    word = word_list[i]
    if word in embedding_matrix_dota.wv.vocab:
        occ+=1
        embedding_matrix.append(np.concatenate((embedding_matrix_wvsg[i], embedding_matrix_dota.wv[word]), 0))
    else:
        embedding_matrix.append(np.concatenate((embedding_matrix_wvsg[i], [0]*50), 0))
print("we have {} words".format(len(word_list)))
print("{} words are in dota dataset".format(occ))
embedding_matrix = np.array(embedding_matrix)
print(embedding_matrix.shape)
EMBEDDING_DIM = embedding_matrix.shape[1]
print(EMBEDDING_DIM)

EMBEDDING_DIM=0
embedding_matrix_smaller = []
occ = 0
occ2 = 0
for i in range(0, len(word_list)):
    word = word_list[i]
    if word in embedding_matrix_dota.wv.vocab:
        occ+=1
        embedding_matrix_smaller.append(np.concatenate((embedding_matrix_wvsg_smaller[i], embedding_matrix_dota.wv[word]), 0))
    else:
        embedding_matrix_smaller.append(np.concatenate((embedding_matrix_wvsg_smaller[i], [0]*50), 0))
print("This one is a smaller embedding matrix")
embedding_matrix_smaller = np.array(embedding_matrix_smaller)
print(embedding_matrix_smaller.shape)
EMBEDDING_DIM = embedding_matrix_smaller.shape[1]
print(EMBEDDING_DIM)

EMBEDDING_DIM=0
embedding_matrix_concat_wiki = []
occ = 0
occ2 = 0
for i in range(0, len(word_list)):
    word = word_list[i]
    if word in wiki_w2v.wv.vocab:
        occ+=1
        embedding_matrix_concat_wiki.append(np.concatenate((embedding_matrix_wvsg[i], embedding_matrix_wiki[i]), 0))
    else:
        embedding_matrix_concat_wiki.append(np.concatenate((embedding_matrix_wvsg[i], [0]*50), 0))
print("we have {} words".format(len(word_list)))
print("{} words are in wiki dataset".format(occ))
embedding_matrix_concat_wiki = np.array(embedding_matrix_concat_wiki)
print(embedding_matrix_concat_wiki.shape)
EMBEDDING_DIM = embedding_matrix_concat_wiki.shape[1]
print(EMBEDDING_DIM)

we have 11243 words
10829 words are in dota dataset
(11243, 150)
150
This one is a smaller embedding matrix
(11243, 100)
100
we have 11243 words
36 words are in wiki dataset
(11243, 150)
150


## 3.5 Dataset for with POS tag

In [ ]:
# combines with pos tag
# but not decode w2v

class MyDataset(Dataset):
    def __init__(self, pos_emb_raw, pos_len, w2v_emb_raw, w2v_matrix):
        self.pos_emb_raw = pos_emb_raw
        self.pos_len = pos_len
        self.w2v_emb_raw = w2v_emb_raw
        self.w2v_matrix = w2v_matrix
        self.w2v_len = w2v_matrix.shape[1]

    def __len__(self):
        return len(self.pos_emb_raw)
    
    def __getitem__(self, idx):
        # put pos tags into one-hot, each word is like [0, 0, 1, ..., 0]
        pos_emb = torch.zeros([len(self.pos_emb_raw[idx]), self.pos_len],dtype=torch.float32).cuda()
        for i in range(len(self.pos_emb_raw[idx])):
            pos_token = torch.zeros([self.pos_len],dtype=torch.float32).cuda()
            pos_token[self.pos_emb_raw[idx]] = 1
            pos_emb[i] = pos_token
        pos_emb.requires_grad_()
        return self.w2v_emb_raw[idx], pos_emb

train_input_index_wpos = MyDataset(pos_emb_train, 50, train_input_index, embedding_matrix)
val_input_index_wpos = MyDataset(pos_emb_val, 50, val_input_index, embedding_matrix)
test_input_index_wpos = MyDataset(pos_emb_test, 50, test_input_index, embedding_matrix)

# 4 - Models

## 4.0 - Some methods

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

## 4.1 - Multihead Attention

In [ ]:
import torch.nn.functional as F
DOT_PRODUCT = 'dot product'
SCALED_DOT_PRODUCT = 'scaled dot product'
GENERAL_1 = 'general-1'
# https://link.springer.com/content/pdf/10.1007/s11063-021-10730-4.pdf
class Multiheadattention(nn.Module):
    def __init__(self, heads, hidden_size, attn_type):
        super().__init__()

        self.hidden_size = hidden_size
        self.heads = heads
        self.head_dim = hidden_size // heads
        self.attn_type = attn_type

        self.WQ = nn.Linear(hidden_size, hidden_size)
        self.WK = nn.Linear(hidden_size, hidden_size)
        self.WV = nn.Linear(hidden_size, hidden_size)

        self.W = nn.Linear(self.head_dim, self.head_dim)

        self.out = nn.Linear(hidden_size, hidden_size)

    def forward(self, q, k, v):
        #seq_len, batch, hidden size
        #q = self.WQ(q)
        #q = q.view(-1, 1, self.heads, self.head_dim)
        #q = q.permute(1, 2, 0, 3)
        q = self.WQ(q).view(-1, 1, self.heads, self.head_dim).permute(1, 2, 0, 3)
        v = self.WV(v).view(-1, 1, self.heads, self.head_dim).permute(1, 2, 0, 3)
        k = self.WK(k).view(-1, 1, self.heads, self.head_dim).permute(1, 2, 0, 3) 
        # seq_len, batch, heads, hidden size// heads, --> batch, heads, seq_len, hidden size// heads

        if self.attn_type == GENERAL_1:
            k = self.W(k)
        # weight_q = self.weight2(q)
        # q_k = self.tanh(torch.matmul(q, k.transpose(-2,-1)) / np.sqrt(self.head_dim))
        # q_k = self.tanh(torch.matmul(q, k.transpose(-2,-1)))
        q_k = torch.matmul(q, k.transpose(-2,-1))

        if self.attn_type == SCALED_DOT_PRODUCT:
            q_k = q_k / np.sqrt(self.head_dim)

        # 1. get rid of tanh
        # 2. add weight_k and weight_q


        q_k = F.softmax(q_k, -1)
        # q_k = self.dropout(q_k)
        out = torch.matmul(q_k, v)
        out = out.permute(2, 0, 1, 3).contiguous().view(-1, 1, self.hidden_size)
        # output = self.tanh(self.out(out))
        output = self.out(out)
        return out

## 4.2 Baseline model (BiLSTM + CRF without Attention) (no POS, twitter25)

In [ ]:
import torch.nn as nn
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, emb_matrix):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

## 4.3 BiLSTM + Multihead Attention model (no POS)

In [ ]:
import torch.nn as nn
class BiLSTM(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, heads, attn_type):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.nb_heads = heads
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.attention = Multiheadattention(heads, hidden_dim, attn_type)


        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim * 2, self.tagset_size)

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def forward(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out_attn = self.attention(lstm_out, lstm_out, lstm_out)
        lstm_out = torch.cat((lstm_out, lstm_out_attn), -1)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim * 2)
        lstm_feats = self.hidden2tag(lstm_out)
        return torch.log_softmax(lstm_feats,-1)

## 4.3 - BiLSTM + CRF + Multihead Attention model

### 4.3.1 BEST MODEL DESIGN: Without POS Tagging embedding

In [ ]:
import torch.nn as nn
class BiLSTM_CRF_Attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, heads, emb_matrix, attn_type):
        super(BiLSTM_CRF_Attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.nb_heads = heads

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.attention = Multiheadattention(self.nb_heads, hidden_dim, attn_type)


        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim * 2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out_attn = self.attention(lstm_out, lstm_out, lstm_out)
        lstm_out = torch.cat((lstm_out, lstm_out_attn), -1)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim*2)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

### 4.3.2 With POS Tagging embedding

In [ ]:
import torch.nn as nn
class BiLSTM_CRF_wpos(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, heads, emb_matrix, attn_type):
        super(BiLSTM_CRF_wpos, self).__init__()
        self.embedding_dim = embedding_dim
        self.nb_heads = heads
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim + 50, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.attention = Multiheadattention(heads, hidden_dim, attn_type)

        # self.multihead_attn = nn.MultiheadAttention(hidden_dim, 4)


        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim * 2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence, pos_emb):
        self.hidden = self.init_hidden()
        w2v_emb = self.word_embeds(sentence)
        embeds = torch.cat((w2v_emb, pos_emb), 1).cuda().view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out_attn = self.attention(lstm_out, lstm_out, lstm_out)
        lstm_out = torch.cat((lstm_out, lstm_out_attn), -1)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim * 2)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, pos, tags):
        feats = self._get_lstm_features(sentence, pos)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence, pos):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence, pos)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

# 5 - Testing and Evaluation

## 5.0 - Preparation + methods

In [ ]:
import numpy as np
import copy

def get_ordered_f1(name, ground_truth, predicted):
    report = classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4, output_dict=True)
    return [name, report["accuracy"], report["T"]["f1-score"], report["S"]["f1-score"], report["C"]["f1-score"], report["D"]["f1-score"], report["P"]["f1-score"], report["O"]["f1-score"]]

def get_ordered_f1_ablation(name, ground_truth, predicted):
    report = classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4, output_dict=True)
    return [name, report["accuracy"]]

def print_table(lis_lis):
    labels_list = ["Model", "T-F1", "T-F1(T)", "T-F1(S)", "T-F1(C)", "T-F1(D)", "T-F1(P)", "T-F1(O)"]
    t = PrettyTable()
    t.field_names = labels_list
    for lis in lis_lis:
        t.add_row(lis)
    print(t)

def print_table_ablation(lis_lis):
    labels_list = ["Model", "T-F1"]
    t = PrettyTable()
    t.field_names = labels_list
    for lis in lis_lis:
        t.add_row(lis)
    print(t)

def cal_acc(model, input_index, output_index):
    predicted = []
    ground_truth = []
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(input_index))):
            input = torch.tensor(input_index[i], dtype=torch.long).cuda()
            outputs = model(input)[1]
            for o in outputs:
                predicted.append(ix_to_tag[o])
            for o in output_index[i]:
                ground_truth.append(ix_to_tag[o])
            labels = torch.tensor(output_index[i], dtype=torch.long).cuda()
            total += labels.size(0)
            outputs = torch.tensor(outputs, dtype=torch.long).cuda()
            correct += (outputs == labels.to(device)).sum().item()
    accuracy = 100 * correct / total
    return predicted, ground_truth, accuracy

def cal_acc_pos(model, input_index, output_index):
    predicted = []
    ground_truth = []
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(input_index))):
            sent_in, pos_in = torch.tensor(input_index[i][0], dtype=torch.long).to(device), input_index[i][1]
            outputs = model(sent_in, pos_in)[1]
            for o in outputs:
                predicted.append(ix_to_tag[o])
            for o in output_index[i]:
                ground_truth.append(ix_to_tag[o])
            labels = torch.tensor(output_index[i], dtype=torch.long).cuda()
            total += labels.size(0)
            outputs = torch.tensor(outputs, dtype=torch.long).cuda()
            correct += (outputs == labels.to(device)).sum().item()
    accuracy = 100 * correct / total
    return predicted, ground_truth, accuracy

def cal_acc_no_CRF(model, input_index, output_index):
    predicted = []
    ground_truth = []
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(input_index))):
            input = torch.tensor(input_index[i], dtype=torch.long).cuda()
            outputs = model(input)
            _,outputs=torch.max(outputs,-1)
            outputs = outputs.tolist()
            for o in outputs:
                predicted.append(ix_to_tag[o])
            for o in output_index[i]:
                ground_truth.append(ix_to_tag[o])
            labels = torch.tensor(output_index[i], dtype=torch.long).cuda()
            total += labels.size(0)
            outputs = torch.tensor(outputs, dtype=torch.long).cuda()
            correct += (outputs == labels.to(device)).sum().item()
    accuracy = 100 * correct / total
    return predicted, ground_truth, accuracy

# set up variables
res_1 = []
res_2 = []
res_5 = []

## 5.1 - Performance Comparison

### 5.1.1 Baseline

In [ ]:
# same as BiLSTM + CRF but model and inputs is diff
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 50
EMBEDDING_DIM = 25
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, embedding_matrix_gt25).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

for name, param in model.state_dict().items():
    param.requires_grad = True

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]


        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)


        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

name = "Baseline"
predicted, ground_truth, _ = cal_acc(model,val_input_index,val_output_index)
res_lis = get_ordered_f1(name, ground_truth, predicted)
res_1.append(res_lis)
print(res_lis)
res_lis = get_ordered_f1_ablation(name, ground_truth, predicted)
res_3.append(res_lis)
print(res_lis)

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 18612.06, train acc: 98.4771, val loss: 2132.10, val acc: 98.3150, time: 574.18s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 4232.67, train acc: 99.4448, val loss: 1303.93, val acc: 99.0946, time: 574.99s


  0%|          | 0/8705 [00:00<?, ?it/s]

['Baseline', 0.9908256880733946, 0.9752066115702479, 0.9895533686601059, 0.97036358081271, 0.9013333333333334, 0.9983479476426483, 0.9924673893073672]
['Baseline', 0.9908256880733946]


### 5.1.2 BiLSTM + CRF + Multihead Attention

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 24
HEADS = 4
EMBEDDING_DIM = 100
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix_smaller, DOT_PRODUCT).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.035, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]


        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

name = "BiLSTM + CRF + MultiHeadAttention"
predicted, ground_truth, _ = cal_acc(model,val_input_index,val_output_index)
res_lis = get_ordered_f1(name, ground_truth, predicted)
res_1.append(res_lis)
print(res_lis)

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 9397.65, train acc: 99.8052, val loss: 811.43, val acc: 99.5173, time: 732.61s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 596.11, train acc: 99.9880, val loss: 898.63, val acc: 99.6312, time: 657.56s


  0%|          | 0/8705 [00:00<?, ?it/s]

['BiLSTM + CRF + MultiHeadAttention', 0.9962823049709181, 0.9833910034602076, 0.9963746223564954, 0.9886118805786397, 0.9859872611464968, 0.999364433710436, 0.9967708907615972]


### 5.1.3 - F1 Comparison

|||||||||
|------|-----|-----|-----|-----|-----|-----|-----|
|Model|T-F1|T-F1(T)|T-F1(S)|T-F1(C)|T-F1(D)|T-F1(P)|T-F1(O)|
|Baseline|99.0826%|97.5207%|98.9553%|97.0364%|90.1333%|99.8348%|99.2467%|
|BiLSTM + CRF|99.6282%|98.3391%|99.6346%|98.8612%|98.5987%|99.9364%|99.6771%|

## 5.2 - Ablation Study: different input embedding model
Notes:
* **Asp** is short for **Aspect**
* **TTV data** is short for **train, test, validation data**

### 5.2.1 Asp 2 only: W2V of train, val, test data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix_wvsg, DOT_PRODUCT).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]


        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

name = "BiLSTM_CRF_W2VTTV"
predicted, ground_truth, _ = cal_acc(model,val_input_index,val_output_index)
res_lis = get_ordered_f1_ablation(name, ground_truth, predicted)
res_2.append(res_lis)

# also for other models
name = "BiLSTM with CRF"
res_lis = get_ordered_f1_ablation(name, ground_truth, predicted)
res_5.append(res_lis)
print(res_lis)

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14598.71, train acc: 99.7360, val loss: 1073.63, val acc: 99.4274, time: 702.38s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 943.71, train acc: 99.9468, val loss: 1173.70, val acc: 99.5803, time: 625.96s


  0%|          | 0/8705 [00:00<?, ?it/s]

['BiLSTM with CRF', 0.9957426395634706]


### 5.2.2 Asp 2 + Asp 3: W2V of TTV data + W2V of Dota game chats

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 150
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, DOT_PRODUCT).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)


        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

name = "BiLSTM_CRF_W2VTTV+DotaChat"
predicted, ground_truth, _ = cal_acc(model,val_input_index,val_output_index)
res_lis = get_ordered_f1_ablation(name, ground_truth, predicted)
res_2.append(res_lis)
print(res_lis)

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 10463.16, train acc: 99.6416, val loss: 1008.56, val acc: 99.3854, time: 651.79s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 1250.48, train acc: 99.9217, val loss: 939.47, val acc: 99.5893, time: 630.53s


  0%|          | 0/8705 [00:00<?, ?it/s]

['BiLSTM_CRF_W2VTTV+DotaChat', 0.9958325837980452]


### 5.2.3 Asp 1 + Asp 2: POS tags + W2V of TTV data

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
EMBEDDING_DIM = 100
heads = 5
model = BiLSTM_CRF_wpos(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, heads, embedding_matrix_wvsg, DOT_PRODUCT).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

for name, param in model.state_dict().items():
    param.requires_grad = True

"""Each epoch will take about 1-2 minutes"""

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index_wpos, total=len(train_input_index_wpos))):
        tags_index = train_output_index[i]

        model.train()
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in, tags_in = torch.tensor(idxs[0], dtype=torch.long).to(device), idxs[1]
        #sentence_in = idxs.to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        output = model(sentence_in, tags_in)
        loss = model.neg_log_likelihood(sentence_in, tags_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()

        loss.backward()
        optimizer.step()

        train_loss+=loss.item()
        model.eval()
        #if i % 1000 == 999 and i < 1100:
        #    _, _, val_acc = cal_acc(model,val_input_index_wpos,val_output_index)
        #    print(f'[{epoch + 1}/{i + 1:5d}] acc: {val_acc:.4f}')
        
    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc_pos(model,train_input_index_wpos,train_output_index)
    _, _, val_acc = cal_acc_pos(model,val_input_index_wpos,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index_wpos, total=len(val_input_index_wpos))):
        tags_index = val_output_index[i]
        sentence_in, tags_in = torch.tensor(idxs[0], dtype=torch.long).to(device), idxs[1]
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, tags_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

name = "BiLSTM_CRF_W2VTTV+POS"
predicted, ground_truth, _ = cal_acc_pos(model,val_input_index_wpos,val_output_index)
res_lis = get_ordered_f1_ablation(name, ground_truth, predicted)
res_2.append(res_lis)
print(res_lis)

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14734.11, train acc: 99.7189, val loss: 1078.22, val acc: 99.4064, time: 761.41s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 896.50, train acc: 99.9558, val loss: 1153.35, val acc: 99.5713, time: 764.46s


  0%|          | 0/8705 [00:00<?, ?it/s]

['BiLSTM_CRF_W2VTTV+POS', 0.9958325837980452]


### 5.2.4 Asp 1 + Asp 2 + Asp 3: POS tags + W2V of TTV data + W2V of Dota game chats
**Note**: this is also included in section 5

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
EMBEDDING_DIM = 150
heads = 5
model = BiLSTM_CRF_wpos(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, heads, embedding_matrix, DOT_PRODUCT).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

for name, param in model.state_dict().items():
    param.requires_grad = True

"""Each epoch will take about 1-2 minutes"""

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index_wpos, total=len(train_input_index_wpos))):
        tags_index = train_output_index[i]

        model.train()
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in, tags_in = torch.tensor(idxs[0], dtype=torch.long).to(device), idxs[1]
        #sentence_in = idxs.to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        output = model(sentence_in, tags_in)
        loss = model.neg_log_likelihood(sentence_in, tags_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()

        loss.backward()
        optimizer.step()

        train_loss+=loss.item()
        model.eval()
        #if i % 1000 == 999 and i < 1100:
        #    _, _, val_acc = cal_acc(model,val_input_index_wpos,val_output_index)
        #    print(f'[{epoch + 1}/{i + 1:5d}] acc: {val_acc:.4f}')
        
    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc_pos(model,train_input_index_wpos,train_output_index)
    _, _, val_acc = cal_acc_pos(model,val_input_index_wpos,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index_wpos, total=len(val_input_index_wpos))):
        tags_index = val_output_index[i]
        sentence_in, tags_in = torch.tensor(idxs[0], dtype=torch.long).to(device), idxs[1]
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, tags_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

name = "BiLSTM_CRF_W2VTTV+POS+DotaChat"
predicted, ground_truth, _ = cal_acc_pos(model,val_input_index_wpos,val_output_index)
res_lis = get_ordered_f1_ablation(name, ground_truth, predicted)
res_2.append(res_lis)
print(res_lis)

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 10951.54, train acc: 99.6577, val loss: 1015.60, val acc: 99.4244, time: 762.63s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 1191.72, train acc: 99.9247, val loss: 890.46, val acc: 99.5383, time: 768.90s


  0%|          | 0/8705 [00:00<?, ?it/s]

['BiLSTM_CRF_W2VTTV+POS+DotaChat', 0.9953528812136475]


### 5.2.5 F1 Comparison

Legend
* Asp1 = POS tag
* Asp2 = W2V of train, test, val data
* Asp3 = Dota game chats

|||
|------|-----|
|Input Embedding|F1|
|Asp2 (W2V TTV)|99.5743%|
|Asp2 (W2V TTV) + Asp3 (Dota chats)|99.5833%|
|Asp1 (POS Tags) + Asp2 (W2V TTV)|99.5833%|
|Asp1 (W2V TTV) + Asp2 (W2V TTV) + Asp3 (Dota chats)|99.5353%|

## 5.3 - Ablation Study: different attention score

In [ ]:
embedding_matrix = embedding_matrix_wvsg

### 5.3.1 - Dot Product

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14732.48, train acc: 99.6777, val loss: 1106.65, val acc: 99.4034, time: 697.42s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 993.01, train acc: 99.9498, val loss: 1160.34, val acc: 99.5773, time: 692.81s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.58625652095701
              precision    recall  f1-score   support

           C     0.9981    0.9750    0.9864      1641
           D     0.9974    0.9598    0.9782       398
           O     0.9931    0.9999    0.9965     18985
           P     1.0000    0.9987    0.9994      3936
           S     0.9997    0.9925    0.9961      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9993    0.9666    0.9827      1469

    accuracy                         0.9959     33354
   macro avg     0.9982    0.9847    0.9913     33354
weighted avg     0.9959    0.9959    0.9958     33354

0.9958625652095701


### 5.3.2 - Scaled Dot Product

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = SCALED_DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14655.11, train acc: 99.6798, val loss: 1124.09, val acc: 99.3884, time: 709.67s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 984.14, train acc: 99.9388, val loss: 1186.80, val acc: 99.5863, time: 713.36s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.55927325058464
              precision    recall  f1-score   support

           C     0.9975    0.9744    0.9858      1641
           D     1.0000    0.9447    0.9716       398
           O     0.9931    0.9998    0.9965     18985
           P     1.0000    0.9987    0.9994      3936
           S     0.9985    0.9928    0.9956      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9951    0.9653    0.9800      1469

    accuracy                         0.9956     33354
   macro avg     0.9977    0.9823    0.9898     33354
weighted avg     0.9956    0.9956    0.9956     33354

0.9955927325058463


### 5.3.3 - General-1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = GENERAL_1
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14791.09, train acc: 99.6376, val loss: 1164.49, val acc: 99.3494, time: 706.28s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 975.19, train acc: 99.9358, val loss: 1162.56, val acc: 99.5593, time: 723.17s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.55927325058464
              precision    recall  f1-score   support

           C     1.0000    0.9738    0.9867      1641
           D     0.9947    0.9397    0.9664       398
           O     0.9930    0.9999    0.9965     18985
           P     0.9997    0.9985    0.9991      3936
           S     0.9982    0.9925    0.9953      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9965    0.9673    0.9817      1469

    accuracy                         0.9956     33354
   macro avg     0.9974    0.9817    0.9894     33354
weighted avg     0.9956    0.9956    0.9956     33354

0.9955927325058463


### 5.3.4 - No Attention

In [ ]:
import torch.nn as nn
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, emb_matrix):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        # print(embeds.size())
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200 
EMBEDDING_DIM = 100
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, embedding_matrix).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14521.27, train acc: 99.6978, val loss: 1069.82, val acc: 99.4124, time: 620.10s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 966.12, train acc: 99.9538, val loss: 1101.40, val acc: 99.5623, time: 623.84s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.56227139173713
              precision    recall  f1-score   support

           C     0.9994    0.9744    0.9867      1641
           D     0.9947    0.9472    0.9704       398
           O     0.9930    0.9999    0.9965     18985
           P     1.0000    0.9985    0.9992      3936
           S     0.9982    0.9919    0.9950      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9979    0.9666    0.9820      1469

    accuracy                         0.9956     33354
   macro avg     0.9976    0.9827    0.9900     33354
weighted avg     0.9956    0.9956    0.9956     33354

0.9956227139173712


### 5.3.5 F1 Comparision

|||
|------|-----|
|Attention Score|F1|
|No Attention|99.5623%|
|Dot Prodcut|99.5863%|
|Scaled Dot Product|99.5593%|
|General-1 |99.5593%|

## 5.4 - Ablation Study: different attention position

In [ ]:
embedding_matrix = embedding_matrix_wvsg

### 5.4.0 - Only Attention

In [ ]:
import torch.nn as nn
class CRF_Attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, heads, emb_matrix, attn_type):
        super(CRF_Attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.nb_heads = heads

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        #print(vocab_size)
        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim * 2, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.attention = Multiheadattention(self.nb_heads, embedding_dim, attn_type)



        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(embedding_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)

        lstm_out = self.attention(embeds, embeds, embeds)
        # lstm_out = torch.cat((embeds, lstm_out_attn), -1)
        # lstm_out, self.hidden = self.lstm(lstm_out, self.hidden)
        #lstm_out = self.layer_norm(lstm_out)
        lstm_out = lstm_out.view(len(sentence), self.embedding_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 17238.78, train acc: 99.5633, val loss: 1131.20, val acc: 99.3104, time: 628.49s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 1589.12, train acc: 99.8765, val loss: 1505.47, val acc: 99.5323, time: 627.14s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.53228998021227
              precision    recall  f1-score   support

           C     0.9981    0.9714    0.9846      1641
           D     0.9770    0.9623    0.9696       398
           O     0.9929    1.0000    0.9965     18985
           P     1.0000    0.9982    0.9991      3936
           S     0.9976    0.9910    0.9943      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9993    0.9612    0.9799      1469

    accuracy                         0.9953     33354
   macro avg     0.9950    0.9834    0.9891     33354
weighted avg     0.9953    0.9953    0.9953     33354

0.9953228998021226


In [ ]:
torch.save(model, 'baseline-only_attention.pt')

### 5.4.1 Attention before LSTM layer

In [ ]:
import torch.nn as nn
class BiLSTM_CRF_Attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, heads, emb_matrix, attn_type):
        super(BiLSTM_CRF_Attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.nb_heads = heads

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim * 2, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.attention = Multiheadattention(self.nb_heads, embedding_dim, attn_type)



        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out_attn = self.attention(embeds, embeds, embeds)
        lstm_out = torch.cat((embeds, lstm_out_attn), -1)
        lstm_out, self.hidden = self.lstm(lstm_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14501.58, train acc: 99.7179, val loss: 1114.73, val acc: 99.4423, time: 782.12s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 833.71, train acc: 99.9528, val loss: 1441.73, val acc: 99.5743, time: 687.92s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.55927325058464
              precision    recall  f1-score   support

           C     0.9988    0.9744    0.9864      1641
           D     0.9947    0.9372    0.9651       398
           O     0.9930    1.0000    0.9965     18985
           P     0.9987    0.9987    0.9987      3936
           S     0.9997    0.9928    0.9962      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9972    0.9653    0.9810      1469

    accuracy                         0.9956     33354
   macro avg     0.9974    0.9812    0.9891     33354
weighted avg     0.9956    0.9956    0.9956     33354

0.9955927325058463


### 5.4.2 Attention after LSTM layer

**This combination is already ran before**

In [ ]:
import torch.nn as nn
class BiLSTM_CRF_Attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, heads, emb_matrix, attn_type):
        super(BiLSTM_CRF_Attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.nb_heads = heads

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.attention = Multiheadattention(self.nb_heads, hidden_dim, attn_type)



        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim * 2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out_attn = self.attention(lstm_out, lstm_out, lstm_out)
        lstm_out = torch.cat((lstm_out, lstm_out_attn), -1)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim*2)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14689.27, train acc: 99.6918, val loss: 1109.53, val acc: 99.4184, time: 680.08s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 970.61, train acc: 99.9408, val loss: 1128.10, val acc: 99.5743, time: 678.43s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.57426395634707
              precision    recall  f1-score   support

           C     0.9994    0.9744    0.9867      1641
           D     0.9947    0.9497    0.9717       398
           O     0.9931    1.0000    0.9966     18985
           P     1.0000    0.9987    0.9994      3936
           S     0.9982    0.9922    0.9952      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9986    0.9666    0.9824      1469

    accuracy                         0.9957     33354
   macro avg     0.9977    0.9831    0.9903     33354
weighted avg     0.9958    0.9957    0.9957     33354

0.9957426395634706


### 5.4.3 No Attention
The model, epoch training output, and T-F1 is same as 5.3.4

### 5.4.3 F1 Comparison

|||
|------|-----|
|Attention Position|F1|
|No Attention|99.5623%|
|Attention only|99.5323%|
|Attention before LSTM layer|99.5593%|
|Attention after LSTM layer|99.5743%|

## 5.5- Ablation Study: different attention heads

In [ ]:
embedding_matrix = embedding_matrix_wvsg

### 5.5.1 - 1 head

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 1
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14512.52, train acc: 99.7269, val loss: 1088.87, val acc: 99.4064, time: 668.42s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 933.85, train acc: 99.9347, val loss: 1189.64, val acc: 99.5563, time: 668.20s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.56227139173713
              precision    recall  f1-score   support

           C     0.9994    0.9732    0.9861      1641
           D     0.9922    0.9548    0.9731       398
           O     0.9929    0.9998    0.9964     18985
           P     0.9990    0.9985    0.9987      3936
           S     0.9997    0.9925    0.9961      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9986    0.9660    0.9820      1469

    accuracy                         0.9956     33354
   macro avg     0.9974    0.9835    0.9903     33354
weighted avg     0.9956    0.9956    0.9956     33354

0.9956227139173712


### 5.5.2 - 2 heads

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 2
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14715.21, train acc: 99.6968, val loss: 1122.05, val acc: 99.4064, time: 666.16s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 914.36, train acc: 99.9337, val loss: 1190.43, val acc: 99.5713, time: 693.13s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.55927325058464
              precision    recall  f1-score   support

           C     0.9988    0.9756    0.9871      1641
           D     0.9974    0.9472    0.9716       398
           O     0.9929    0.9999    0.9964     18985
           P     0.9997    0.9987    0.9992      3936
           S     0.9979    0.9925    0.9952      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9993    0.9626    0.9806      1469

    accuracy                         0.9956     33354
   macro avg     0.9980    0.9824    0.9900     33354
weighted avg     0.9956    0.9956    0.9956     33354

0.9955927325058463


### 5.5.3 - 5 heads

This combination is already ran before

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14755.55, train acc: 99.7249, val loss: 1094.87, val acc: 99.4483, time: 715.76s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 939.56, train acc: 99.9528, val loss: 1125.88, val acc: 99.5863, time: 713.44s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.5892546621095
              precision    recall  f1-score   support

           C     0.9994    0.9744    0.9867      1641
           D     1.0000    0.9623    0.9808       398
           O     0.9930    0.9999    0.9965     18985
           P     0.9997    0.9985    0.9991      3936
           S     1.0000    0.9928    0.9964      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9993    0.9666    0.9827      1469

    accuracy                         0.9959     33354
   macro avg     0.9988    0.9849    0.9917     33354
weighted avg     0.9959    0.9959    0.9959     33354

0.9958925466210949


### 5.5.4 - 10 heads

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 10
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14835.09, train acc: 99.7109, val loss: 1080.03, val acc: 99.4423, time: 671.56s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 910.37, train acc: 99.9438, val loss: 1070.96, val acc: 99.5743, time: 667.10s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.56826767404209
              precision    recall  f1-score   support

           C     0.9994    0.9738    0.9864      1641
           D     0.9974    0.9573    0.9769       398
           O     0.9929    0.9999    0.9964     18985
           P     1.0000    0.9987    0.9994      3936
           S     0.9985    0.9922    0.9953      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9993    0.9646    0.9816      1469

    accuracy                         0.9957     33354
   macro avg     0.9982    0.9838    0.9909     33354
weighted avg     0.9957    0.9957    0.9957     33354

0.995682676740421


### 5.5.5 F1 Comparision

|||
|------|-----|
|Number of heads|F1|
|1 head|99.5623%|
|2 heads|99.5593%|
|5 heads|99.5893%|
|10 heads|99.5683%|

## 5.6 - Ablation Study: different stacked layer 

In [ ]:
embedding_matrix = embedding_matrix_wvsg

### 5.6.1 - 1 layer

In [ ]:
import torch.nn as nn
class BiLSTM_CRF_Attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, heads, emb_matrix, attn_type):
        super(BiLSTM_CRF_Attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.nb_heads = heads

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        #print(vocab_size)
        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim * 2, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.attention = Multiheadattention(self.nb_heads, embedding_dim, attn_type)



        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out_attn = self.attention(embeds, embeds, embeds)
        lstm_out = torch.cat((embeds, lstm_out_attn), -1)
        lstm_out, self.hidden = self.lstm(lstm_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 14501.58, train acc: 99.7179, val loss: 1114.73, val acc: 99.4423, time: 782.12s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 833.71, train acc: 99.9528, val loss: 1441.73, val acc: 99.5743, time: 687.92s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.55927325058464
              precision    recall  f1-score   support

           C     0.9988    0.9744    0.9864      1641
           D     0.9947    0.9372    0.9651       398
           O     0.9930    1.0000    0.9965     18985
           P     0.9987    0.9987    0.9987      3936
           S     0.9997    0.9928    0.9962      3322
        SEPA     1.0000    1.0000    1.0000      3603
           T     0.9972    0.9653    0.9810      1469

    accuracy                         0.9956     33354
   macro avg     0.9974    0.9812    0.9891     33354
weighted avg     0.9956    0.9956    0.9956     33354

0.9955927325058463


### 5.6.2 - 2 layers

In [ ]:
import torch.nn as nn
class BiLSTM_CRF_Attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, heads, emb_matrix, attn_type):
        super(BiLSTM_CRF_Attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.nb_heads = heads

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim * 2, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        
        self.attention = Multiheadattention(self.nb_heads, embedding_dim, attn_type)



        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out_attn = self.attention(embeds, embeds, embeds)
        lstm_out = torch.cat((embeds, lstm_out_attn), -1)
        lstm_out, self.hidden = self.lstm(lstm_out, self.hidden)
        lstm_out, self.hidden = self.lstm2(lstm_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 22846.54, train acc: 98.7853, val loss: 1842.15, val acc: 98.6089, time: 807.85s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 2505.09, train acc: 99.5964, val loss: 1448.19, val acc: 99.3074, time: 700.06s


  0%|          | 0/8705 [00:00<?, ?it/s]

99.30143311147089
              precision    recall  f1-score   support

           C     0.9962    0.9567    0.9761      1641
           D     0.9639    0.9397    0.9517       398
           O     0.9924    0.9978    0.9951     18985
           P     0.9919    0.9987    0.9953      3936
           S     0.9943    0.9910    0.9926      3322
        SEPA     0.9997    1.0000    0.9999      3603
           T     0.9895    0.9578    0.9734      1469

    accuracy                         0.9930     33354
   macro avg     0.9897    0.9774    0.9834     33354
weighted avg     0.9930    0.9930    0.9930     33354

0.9930143311147088


### 5.6.3 - 3 layers

In [ ]:
import torch.nn as nn
class BiLSTM_CRF_Attention(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, heads, emb_matrix, attn_type):
        super(BiLSTM_CRF_Attention, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.nb_heads = heads

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.word_embeds.weight.requires_grad = True
        
        self.lstm = nn.LSTM(embedding_dim * 2, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        self.lstm3 = nn.LSTM(hidden_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)
        
        
        self.attention = Multiheadattention(self.nb_heads, embedding_dim, attn_type)



        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out_attn = self.attention(embeds, embeds, embeds)
        lstm_out = torch.cat((embeds, lstm_out_attn), -1)
        lstm_out, self.hidden = self.lstm(lstm_out, self.hidden)
        lstm_out, self.hidden = self.lstm2(lstm_out, self.hidden)
        lstm_out, self.hidden = self.lstm3(lstm_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
HEADS = 5
EMBEDDING_DIM = 100
attention_type = DOT_PRODUCT
model = BiLSTM_CRF_Attention(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, HEADS, embedding_matrix, attention_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]
        model.zero_grad()
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss.backward()
        optimizer.step()
        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

predicted, ground_truth, accuracy = cal_acc(model, val_input_index,val_output_index)
print(accuracy)
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import numpy as np
print(classification_report(y_true=np.array(ground_truth), y_pred=np.array(predicted), digits=4))
print(f1_score(y_true=np.array(ground_truth), y_pred=np.array(predicted), average='micro'))

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 29675.67, train acc: 96.9913, val loss: 3512.43, val acc: 96.9389, time: 726.53s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 6711.90, train acc: 98.1528, val loss: 2475.92, val acc: 97.8563, time: 727.26s


  0%|          | 0/8705 [00:00<?, ?it/s]

97.88930862865024
              precision    recall  f1-score   support

           C     0.8463    0.8793    0.8625      1641
           D     0.7742    0.6633    0.7145       398
           O     0.9922    0.9968    0.9945     18985
           P     0.9904    0.9964    0.9934      3936
           S     0.9770    0.9868    0.9819      3322
        SEPA     1.0000    0.9994    0.9997      3603
           T     0.9234    0.8285    0.8733      1469

    accuracy                         0.9789     33354
   macro avg     0.9291    0.9072    0.9171     33354
weighted avg     0.9785    0.9789    0.9785     33354

0.9788930862865024


### 5.6.4 F1 Comparision

|||
|------|-----|
|Number of layers|F1|
|1 BiLSTM layer|99.5593%|
|2 BiLSTM layers|99.3014%|
|3 BiLSTM layers|97.8893%|


## 5.7 - Ablation Study: with/without CRF

### 5.7.1 BiLSTM without CRF

In [ ]:
# rebuild this embedding matrix that might have been overwritten by running sections 5.X
embedding_matrix = embedding_matrix_wvsg

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 200
EMBEDDING_DIM = 100
heads = 5
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, heads, DOT_PRODUCT).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

"""Each epoch will take about 1-2 minutes"""

import datetime
from tqdm.notebook import tqdm

for epoch in range(2):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(tqdm(train_input_index, total=len(train_input_index))):
        tags_index = train_output_index[i]

        model.train()
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        #sentence_in = idxs.to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        output = model(sentence_in)
        loss=criterion(output,targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()

        loss.backward()
        optimizer.step()

        train_loss+=loss.item()
        model.eval()
        #if i % 1000 == 999:
        #    _, _, val_acc = cal_acc_no_CRF(model,val_input_index,val_output_index)
        #    print(f'[{epoch + 1}/{i + 1:5d}] acc: {val_acc:.4f}')
        
    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc_no_CRF(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc_no_CRF(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(tqdm(val_input_index, total=len(val_input_index))):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        output = model(sentence_in)
        loss=criterion(output,targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

name = "BiLSTM without CRF"
predicted, ground_truth, _ = cal_acc_no_CRF(model,val_input_index, val_output_index)
res_lis = get_ordered_f1_ablation(name, ground_truth, predicted)
res_5.append(res_lis)
print(res_lis)

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/26078 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:1, Training loss: 9682.00, train acc: 97.0957, val loss: 1002.52, val acc: 97.0258, time: 140.50s


...

  0%|          | 0/8705 [00:00<?, ?it/s]

  0%|          | 0/8705 [00:00<?, ?it/s]

Epoch:2, Training loss: 1745.88, train acc: 99.2682, val loss: 497.20, val acc: 99.0226, time: 124.97s


  0%|          | 0/8705 [00:00<?, ?it/s]

['BiLSTM without CRF', 0.990705762427295]


### 5.7.2 F1 Comparison

|||
|------|-----|
|Model|F1|
|BiLSTM with CRF|99.5743%|
|BiLSTM without CRF|99.0706%|